In [1]:
import pandas as pd
import math
import matplotlib.pyplot as plt
from mpl_toolkits.basemap import Basemap
import numpy as np
import xarray as xr

In [117]:
# open catalogue dataset
df = pd.read_csv('/home/data/ReAnalysis/ERA5/Storm_analysis/NAECv1/NAEC_1979_2020_v1.csv')

# open catalogue that contains storm that were active for more than 24h
# more in CRCM6 domain
df24 = pd.read_csv('/pampa/cloutier/etc_24_nna.csv')

# open netcdf mask file
file = '/pampa/picart/Masks/mask_GEM5_ERA5grid'
data = xr.open_dataset(file)

# export netcdf to dataframe
mask = data.to_dataframe()

# drop index lat lon, but keep columns
mask = mask.reset_index()

# keep in mask values where HU = true
maskT = mask.loc[mask.HU != False]

# open catalogue that contains storm that were active for more than 24h in CRCM domain 
# for each season
djf = pd.read_csv('/pampa/cloutier/etc_24_nna_djf.csv')
mam = pd.read_csv('/pampa/cloutier/etc_24_nna_mam.csv')
jja = pd.read_csv('/pampa/cloutier/etc_24_nna_jja.csv')
son = pd.read_csv('/pampa/cloutier/etc_24_nna_son.csv')

### Storm track density

In [2]:
# Define function to calculate distance between two points using Haversine formula
# Haversine package ?
def calculate_distance(lat1, lon1, lat2, lon2):
    # Earth radius in kilometers
    earth_radius = 6371

    # Convert latitude and longitude to radians
    lat1_rad = math.radians(lat1)
    lon1_rad = math.radians(lon1)
    lat2_rad = math.radians(lat2)
    lon2_rad = math.radians(lon2)

    # Calculate differences
    dlat = lat2_rad - lat1_rad
    dlon = lon2_rad - lon1_rad

    # Haversine formula
    a = math.sin(dlat / 2) ** 2 + math.cos(lat1_rad) * math.cos(lat2_rad) * math.sin(dlon / 2) ** 2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))

    # Calculate distance in kilometers
    distance = earth_radius * c

    return distance

In [374]:
r = 2.25
i = 0
season = [djf, mam, jja, son]
season_txt = ['djf', 'mam', 'jja', 'son']

for m in season : 
    
    m_density = {} # will contain the grid point and density

    # iterate through all grid point
    for _, row in m.iterrows() : 
        track = {} #  will contain the year and storm id of the grid points found
        lat1 = row['latitude'] 
        lon1 = row['longitude'] 
        
        # restricting the research in a 2.5 x 2.5deg square
        neighbors = m[
            (m['latitude'] >= lat1 - r) & (m['latitude'] <= lat1 + r) &
            (m['longitude'] >= lon1 - r) & (m['longitude'] <= lon1 + r)
            ]

        # Find the grid points that are within a 250km radius 
        for _, row in neighbors.iterrows():
            lat2 = row['latitude'] 
            lon2 = row['longitude'] 
            year = str(row['datetime'])[:4]
            storm_id = row['storm']

            # calculate distance between grid points
            distance = calculate_distance(lat1, lon1, lat2, lon2)
            
            # add year and storm_id of grid points that are within the radius
            if distance <= 250 : 
                track['year'] = track.get('year', []) + [year]
                track['storm_id'] = track.get('storm_id', []) + [storm_id]

        # turn track dictionnary into dataframe and count the unique occurence of 
        # every storm for each year
        track_df = pd.DataFrame(data = track)
        track_df = track_df.groupby('year')['storm_id'].nunique()

        # number of storm per season in average
        average = track_df.mean()
        
        # add coord and average density
        m_density['lat'] = m_density.get('lat', []) + [lat1]
        m_density['lon'] = m_density.get('lon', []) + [lon1]
        m_density['avg_per_season'] = m_density.get('avg_per_season', []) + [average]

    # transfort m_density into dataframe and save as csv
    df_density = pd.DataFrame(data = m_density)
    df_density.to_csv('/pampa/cloutier/' + season_txt[i] + '_density.csv')
    i += 1

KeyboardInterrupt: 

In [371]:
season_txt = ['djf', 'mam', 'jja', 'son']
for i in (0,1,2,3) : 
    print(season_txt[i])

djf
mam
jja
son


In [364]:
# test de la densité avec seulement SON
r = 2.25
son_density = {} # will contain the grid point and density

# iterate through all grid point
for _, row in son.iterrows() : 
    track = {} #  will contain the year and storm id of the grid points found
    lat1 = row['latitude'] 
    lon1 = row['longitude'] 

    # restricting the research in a 2.5 x 2.5deg square
    neighbors = son[
        (son['latitude'] >= lat1 - r) & (son['latitude'] <= lat1 + r) &
        (son['longitude'] >= lon1 - r) & (son['longitude'] <= lon1 + r)
        ]

    # Find the grid points that are within a 250km radius 
    for _, row in neighbors.iterrows():
        lat2 = row['latitude'] 
        lon2 = row['longitude'] 
        year = str(row['datetime'])[:4]
        storm_id = row['storm']

        # calculate distance between grid points
        distance = calculate_distance(lat1, lon1, lat2, lon2)

        # add year and storm_id of grid points that are within the radius
        if distance <= 250 : 
            track['year'] = track.get('year', []) + [year]
            track['storm_id'] = track.get('storm_id', []) + [storm_id]

    # turn track dictionnary into dataframe and count the unique occurence of 
    # every storm for each year
    track_df = pd.DataFrame(data = track)
    track_df = track_df.groupby('year')['storm_id'].nunique()

    # number of storm per season in average
    average = track_df.mean()

    # add coord and average density
    son_density['lat'] = son_density.get('lat', []) + [lat1]
    son_density['lon'] = son_density.get('lon', []) + [lon1]
    son_density['avg_per_season'] = m_density.get('avg_per_season', []) + [average]

# transfort m_density into dataframe and save as csv
df_density = pd.DataFrame(data = son_density)

df.density_to.csv('/pampa/cloutier/test_son_density.csv')

KeyboardInterrupt: 

In [98]:
df = pd.read_csv('/pampa/cloutier/density.csv')
#mam = df.loc[(df.season == 'MAM') & (df.latitude < 50) & (df.longitude < 250)]

In [99]:
mam = df.loc[df.season == 'MAM']

In [100]:
mam

,Unnamed: 0,latitude,longitude,season,avgVORSmax,storm_count
29,29,20.00,287.00,MAM,0.000111,1
114,114,20.25,286.75,MAM,0.000111,1
188,188,20.50,287.00,MAM,0.000111,1
291,291,20.75,287.50,MAM,0.000111,1
382,382,21.00,287.25,MAM,0.000111,1
...,...,...,...,...,...,...
193785,193785,80.00,227.00,MAM,0.000079,1
193791,193791,80.00,268.00,MAM,0.000096,1
193798,193798,80.00,284.75,MAM,0.000088,1
193804,193804,80.00,306.25,MAM,0.000078,1


In [61]:
r = 2.25
i = 0
#season = [djf, mam, jja, son]
#season_txt = ['djf', 'mam', 'jja', 'son']

m_density = {} # will contain the grid point and density

# iterate through all grid point
for _, row in mam.iterrows() : 
    track = {} #  will contain the avgvors and storm_count of the grid points found
    lat1 = row['latitude'] 
    lon1 = row['longitude'] 

    # restricting the research in a 2.5 x 2.5deg square
    neighbors = mam[
        (mam['latitude'] >= lat1 - r) & (mam['latitude'] <= lat1 + r) &
        (mam['longitude'] >= lon1 - r) & (mam['longitude'] <= lon1 + r)
        ]

    # Find the grid points that are within a 250km radius 
    for _, row in neighbors.iterrows():
        lat2 = row['latitude'] 
        lon2 = row['longitude'] 
        storm_count = row['storm_count']
        avgvors = row['avgVORSmax']

        # calculate distance between grid points
        distance = calculate_distance(lat1, lon1, lat2, lon2)

        # add storm_count and avgVORSmax of grid points that are within the radius
        if distance <= 250 : 
            
            print(distance)
            m_density['storm_count'] = m_density.get('storm_count', []) + [storm_count]
         
        # déterminer la moyenne de tous les points de grille trouvés
        
    


0.0
35.86836121807902
130.17246277746148
179.91960670916686
239.78366894828858
35.86836121807902
0.0
94.78603425738302
146.06759867244827
205.84808917520573
130.17246277746148
94.78603425738302
0.0
55.597463322278735
113.4037111549732
168.26322166512062
166.79238996683762
168.26322166512003
194.5911216279777
195.8490890228992
223.4877002427163
179.91960670916686
146.06759867244827
55.597463322278735
0.0
59.86615068133786
113.37512516902041
111.19492664455889
113.37512516901953
138.993658305699
140.73805281927096
168.24390830389083
195.83247564331032
194.59112162797842
199.5101921636948
213.58861912812174
235.1869125322453
222.38985328911778
223.4731237003567
226.6918703438246
0.0
72.07020265220436
226.91074752567246
212.62442939773132
199.73929249152167
179.35198026477983
172.48738754825297
213.46629444572966
205.42196699019001
194.5911216279777
239.78366894828858
205.84808917520573
113.4037111549732
59.86615068133786
0.0
55.597463322279445
59.81185144715277
70.96895855777807
86.253491

222.38985328911778
166.79238996683833
168.20500916010099
172.373411179226
140.67992422887363
145.62159756551307
111.19492664455889
83.39619498341881
86.15841382741401
105.68834695538224
136.6141394405319
172.92552353547745
59.64719181734515
70.41244679760025
85.38994605543543
27.798731661140078
35.18289061486716
51.31307042625416
70.41551759903558
90.62979603902173
111.351923872352
0.0
64.58121536134291
86.10809573076251
129.1613232113992
150.68758826671893
35.13590241847873
27.798731661139367
35.1359024184814
152.96542636273693
174.1389061628022
195.3806373675187
70.22449296999311
59.59177111969344
85.04097822965208
160.1104145326619
105.26544693563514
93.74785165878626
119.53997635071063
171.51536021519354
186.39995572792358
186.39995572792358
188.9493599399733
175.20314095513157
163.09659120795845
153.0194924479319
210.09922627458232
187.2805686229906
210.09922627458232
223.6862649220841
244.9463655275414
204.77074959533215
226.40073448054747
231.83018774564567
179.10526011129238
17

221.53761130708332
189.89299901822488
153.3112420948852
153.3112420948852
163.5829506667106
175.9101959367584
205.19309219972098
221.53761130708673
119.23856584721453
111.19492664455889
170.42795397893582
187.2680702657066
210.60464225789283
191.06381520449668
200.89142779812877
160.1104145326619
140.1939764670469
59.59177111969502
85.04097822965208
120.80327870728635
160.1104145326619
131.44077820770553
110.60695524186296
90.0440708065602
69.9915605534194
51.05462745099125
69.9915605534194
110.60695524186715
173.52959806085997
128.23546922375257
21.37282084725862
0.0
21.37282084726297
42.74560009989555
128.23546922375257
149.60741652399304
192.3503958750596
194.00240569940775
130.98596570977065
27.798731661139367
172.91712779580328
139.34001977908724
86.0623241897435
119.09503264067115
185.52494775354467
169.0215005500702
174.7282787646274
140.59755897443844
152.82390000850248
235.88272895162618
209.52855437913195
197.45428814961525
186.99620564797584
178.43879188757984
172.0659338670

231.38118036956283
238.13977746254133
246.55723833645257
204.77074959533215
232.65174900052932
244.94636552754403
172.10461666267398
178.52270952562571
187.13854494955157
140.5975589744391
145.3030613152208
188.31509834959027
111.19492664455889
113.18614702589534
185.0849118644085
207.38732774584452
118.64707657499815
157.54751352497604
84.15721163336725
69.74970225810246
59.45220422002544
138.04426778107936
68.91427708021394
34.85104024539188
191.20327280024847
83.91941795130552
0.0
108.3270570712174
34.803102980020256
69.5579521997166
150.39630833196205
167.23525062973314
118.73521924834057
183.30839576109074
152.33150638341138
208.08844640715373
186.2801308451606
237.24071418597717
246.55723833645257
238.1397774625398
231.38118036956283
221.71657371820822
212.3510525866608
199.17965542921178
195.74834134602816
232.65174900052932
237.9565505628885
223.3219447683188
168.13618491204764
166.79238996683833
168.1361849120471
172.10461666267292
178.52270952562571
237.9565505628854
235.8827

231.87736373114538
243.9448613752261
178.27049592447327
171.98838453236112
140.56197444817533
152.52901325916372
162.27775759137248
174.01105359596306
127.65312325234116
152.58983687439058
93.24924768676986
150.39630833196205
59.36784997717068
155.98105066538514
168.54241322422192
0.0
27.798731661140078
167.9051274470619
194.05397264395106
83.39619498341952
237.80408161582935
222.38985328911707
246.05045593442122
204.55104342433094
194.5911216279777
231.87736373114768
166.79238996683762
162.27775759137248
201.9942528594123
113.14191445594585
111.19492664455818
85.9654026728618
133.53977809038918
205.42177417125222
195.44738281224926
55.597463322279445
59.36784997717216
69.46170913918432
83.61955114790376
100.13203476376668
118.01478223569644
168.54241322422192
27.798731661139367
107.55234913163129
127.73829871581536
148.08815312840463
168.5424132242261
0.0
186.6497206204606
167.9051274470619
68.03504423037232
87.33752166536688
147.53284857241437
154.92504248483647
194.05397264395512
14

...

241.56720080018957
234.84392418521662
223.18860361196118
222.38985328911778
246.0433695457963
224.9862355247983
208.64766271514392
195.49927900340782
194.5911216279777
212.36101900057488
191.4025480574462
176.0489403799232
170.96831387338037
167.84611408306682
166.79238996683833
218.50737509302647
191.09116060874356
178.75573297091728
143.95518538501108
140.2504992807092
138.9936583056983
133.9895466688071
124.53144071993673
117.30962456096906
112.75469701224947
111.19492664455889
125.0839574113251
111.88053679218282
100.41789987268918
85.4551078788837
176.39153152532052
141.57568524660624
92.88045966685165
66.89557492524253
55.597463322279445
169.294159338844
151.02270974930312
132.82924155903416
96.85309281121778
79.25764596703688
62.222213514533244
46.3684727019509
148.08114116126302
129.57185676693362
111.06222905956501
92.55230709295866
74.04213991671654
55.531776576954016
37.021266116990546
18.510657578062776
0.0
168.49080896855446
150.3111790815329
132.20987335856728
96.41974775

249.2959761349116
234.72940568467797
223.18107412704805
222.38985328911707
234.64714660608897
215.95860956979018
202.54337386151855
198.16487270479567
195.49068665177347
194.5911216279777
236.8705963703062
211.97324854132867
200.97917310493818
170.9290264217439
167.83611036518528
166.79238996683762
157.73682836149771
149.82555030433215
143.90854424365253
140.23853241434156
138.9936583056983
144.95862710605448
133.7890808578161
124.41014603138522
112.73981843866609
186.65742102674412
154.35267971487093
111.64049152076649
91.28038331383262
83.39619498341881
175.62026551549565
158.17218163530313
140.99445446240568
107.96642251270983
92.59130950522838
78.57965936738013
66.79528889654284
150.3111790815329
132.20987335856728
114.2243725898214
96.41974775267653
78.91876776854306
61.979451345065144
46.22374495097385
33.37269344274218
27.798731661139367
165.77399667686552
147.35577854236556
128.93716691925113
92.09895999500999
36.83992825556214
0.0
131.58823423829276
61.73598669913406
46.078712

210.80774460853456
177.40051246560336
133.1870431549417
66.49415514865035
0.0
45.64187200616828
121.7545905459709
248.56809082622038
223.15852092318784
222.38985328911778
223.15852092318784
194.5911216279777
195.46495127597277
202.31971209766684
215.37545657423652
233.59426196526687
166.79238996683833
167.8061491040418
140.20269241518446
112.6952573433587
27.798731661140078
0.0
33.17187256969886
33.17187256969886
61.001481574564416
58.45517859067858
77.61717712728779
91.1366325105591
106.01484767691719
154.7559071040045
234.38607482616234
225.44879168947094
225.44879168947034
248.56809082622038
244.29164917870924
223.90992828341078
194.5911216279777
167.8061491040418
170.8113199785602
200.07649702278582
210.80774460853456
235.14491959085998
143.76879033701653
149.5233645112787
157.22624222166763
177.40051246560336
216.0080400615586
124.04625695863602
169.77384837655669
184.01438306603092
83.39619498341952
85.37670571690684
91.06013338452097
99.81589323873368
123.73826535425412
152.7507

229.14927532759427
225.41907887944998
222.38985328911707
225.41907887944933
234.27173535699097
202.24524958573133
198.0294987252141
170.77214649327652
143.72227413377755
77.81015231784978
61.001481574564416
72.21479810247465
36.10759779841233
0.0
45.495705001263254
27.798731661140078
33.12163644257589
45.495705001260355
94.23069188678937
164.42923514863307
223.15101684935144
194.5911216279777
195.4563888913081
215.18110328651704
223.6406870884036
243.85280491703895
167.79618118834165
170.77214649327652
175.62002532487352
190.30488001607202
222.0660693576436
234.56805375799377
143.72227413377755
177.06096601622437
188.93133057116765
215.3803022377956
112.68043230826937
111.19492664455889
112.68043230826937
117.02384611495971
132.9862000206387
156.0012342714942
183.43243742537052
198.2432375349705
85.35714830426129
90.9867652525116
137.35928994073686
58.483677507350805
172.50914831883873
33.17187256969886
27.798731661139367
94.67143722429675
129.70984536553664
165.2474649555525
0.0
18.05

234.27173535699097
229.14927532759427
225.41907887944998
222.38985328911707
243.85280491703674
223.6406870884036
202.24524958573133
195.4563888913081
195.4563888913081
222.06606935764114
210.418604771825
199.77531454069774
182.1904008921278
175.62002532487352
170.77214649327652
167.79618118834165
215.3803022377956
188.93133057116765
177.06096601622437
166.35631344595348
157.05603905961212
149.42271357527392
143.72227413377755
140.1907689906556
138.9936583056983
140.1907689906556
183.43243742536754
169.29099903000804
156.0012342714942
132.9862000206387
112.68043230826937
112.68043230826937
137.35928994073686
90.9867652525116
85.35714830426129
83.39619498341881
138.6515073839228
55.597463322279445
58.483677507350805
165.2474649555525
129.70984536553664
112.09872226719605
77.55227905603931
61.001481574564416
45.641872006169734
33.17187256969886
27.798731661139367
162.47845102001156
108.32120339668839
90.26812497193548
72.21479810247465
54.161272481212
18.05382374235301
0.0
164.42923514863

In [60]:
print(lat1, lon1)
print(df_density)

49.75 249.75
       storm_count
0                1
1                1
2                1
3                1
4                1
...            ...
21676            2
21677            1
21678            2
21679            2
21680            1

[21681 rows x 1 columns]


In [101]:
# test de la densité avec seulement SON
r = 2.25
mam_density = {} # will contain the grid point and density

# iterate through all grid point
for _, row in mam.iterrows() : 
    track = {} #  will contain the year and storm id of the grid points found
    lat1 = row['latitude'] 
    lon1 = row['longitude'] 

    # restricting the research in a 2.5 x 2.5deg square
    neighbors = mam[
        (mam['latitude'] >= lat1 - r) & (mam['latitude'] <= lat1 + r) &
        (mam['longitude'] >= lon1 - r) & (mam['longitude'] <= lon1 + r)
        ]

    # Find the grid points that are within a 250km radius 
    for _, row in neighbors.iterrows():
        lat2 = row['latitude'] 
        lon2 = row['longitude'] 
        storm_count = row['storm_count']
        avgvors = row['avgVORSmax']

        # calculate distance between grid points
        distance = calculate_distance(lat1, lon1, lat2, lon2)

        # add year and storm_id of grid points that are within the radius
        if distance <= 250 : 
            track['storm_count'] = track.get('storm_count', []) + [storm_count]

    # turn track dictionnary into dataframe and count the unique occurence of 
    # every storm for each year
    track_df = pd.DataFrame(data = track)
    #track_df = track_df.groupby('year')['storm_id'].nunique()

    # number of storm per season in average
    average = track_df['storm_count'].mean()

    # add coord and average density
    mam_density['lat'] = mam_density.get('lat', []) + [lat1]
    mam_density['lon'] = mam_density.get('lon', []) + [lon1]
    mam_density['storm_count'] = mam_density.get('storm_count', []) + [average]
    mam_density['avgvors'] = mam_density.get('avgvors', []) + [avgvors]

# transfort m_density into dataframe and save as csv
df_density = pd.DataFrame(data = mam_density)

#df.density_to.csv('/pampa/cloutier/test_son_density.csv')

In [104]:
df_density

,lat,lon,storm_count,avgvors
0,20.00,287.00,1.000000,0.000111
1,20.25,286.75,1.000000,0.000111
2,20.50,287.00,1.000000,0.000111
3,20.75,287.50,1.000000,0.000111
4,21.00,287.25,1.047619,0.000111
...,...,...,...,...
49177,80.00,227.00,1.000000,0.000079
49178,80.00,268.00,1.000000,0.000096
49179,80.00,284.75,1.055556,0.000088
49180,80.00,306.25,1.000000,0.000078


In [109]:
df_density.to_csv('/pampa/cloutier/test_densite_mam.csv')

In [115]:
import pandas as pd
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import numpy as np                         
import matplotlib.pyplot as plt            
import matplotlib.colors as colors
import os                                  
import rpnpy.librmn.all as rmn             
import rpn_funcs_chris
from rpn_funcs_chris import open_var_2d, lambert_map
from matplotlib.path import Path
import matplotlib.patches as patches
import netCDF4

def get_verts_new(categ_box):
    verts = [(categ_box['west'], categ_box['south']),
     (categ_box['west'], categ_box['north']),
     (categ_box['east'], categ_box['north']),
     (categ_box['east'], categ_box['south']),
     (0., 0.)]
    return verts

box = {'west':-180, 'east':0, 'north':90, 'south':20}


Dom=netCDF4.Dataset('./2p5kmDomain/var_MG_free_grid_date.nc3','r')
latd = Dom.variables['lat'][:,:]
lond = Dom.variables['lon'][:,:]
#sftlf = Domainmask.variables['sftlf'][:,:]

Track=netCDF4.Dataset('../data/ERA5_trackedstorm_output/Track_density_2000_2020_all_radius_400km_DJF.nc','r')

lat = Track.variables['lat'][:,:]
lon = Track.variables['lon'][:,:]
den = Track.variables['trackDen'][:,:]/21./3.

#norm = colors.BoundaryNorm(boundaries=[1,400,800,1200,1600,2000,2400,2800], ncolors=256)

fig, ax = lambert_map(extent=(-160, -11, 5, 90), figsize=(10,10))

cf = ax.contourf(lon, lat, den, transform=ccrs.PlateCarree(), zorder=6,cmap=plt.cm.get_cmap('Spectral_r'),
                 levels=np.arange(0,10,1), extend='max')
ax.gridlines(zorder=21)

verts = get_verts_new(box)
codes = [Path.MOVETO, Path.LINETO, Path.LINETO, Path.LINETO, Path.CLOSEPOLY,]
path = Path(verts, codes)
import matplotlib.patheffects as path_effects
patch = patches.PathPatch(path, facecolor='red', lw=3, transform=ccrs.PlateCarree(), zorder=20, alpha=0.3)
patch = patches.PathPatch(path, facecolor='none', ec='white', lw=3,
                          transform=ccrs.PlateCarree(), zorder=20)
ax.add_patch(patch)

#cb = plt.colorbar(cf, shrink=0.5) #, ticks=np.arange(0,3,1))
cb = plt.colorbar(cf, shrink=0.4) #, ticks=np.arange(0,3,1))
cb.ax.tick_params(labelsize=8)
#cb.ax.set_yticklabels('numbers')
#plt.savefig('ERA5_storm_track_density_2000_2020all_R400km_DJF_permonth', bbox_inches='tight', dpi=200)
#plt.savefig('ERA5_storm_track_density_2000_2020all_zoomin', bbox_inches='tight', dpi=200)
plt.show()


SyntaxError: invalid syntax (__init__.py, line 100)